In [2]:
from pymongo import MongoClient
import pandas as pd
import folium

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db, coll = connectCollection('companies','companies')

In [7]:
companies=list(coll.find())

df=pd.DataFrame(companies)

In [8]:
print(set(df.category_code))

{'network_hosting', 'enterprise', 'sports', 'local', 'health', 'medical', 'news', 'automotive', 'web', 'manufacturing', 'semiconductor', 'software', 'other', 'design', 'games_video', 'government', 'messaging', 'social', 'ecommerce', 'advertising', 'legal', 'photo_video', 'mobile', 'consulting', 'search', 'real_estate', 'analytics', 'biotech', 'cleantech', 'nonprofit', 'hardware', 'travel', 'public_relations', 'hospitality', 'music', None, 'education', 'finance', 'fashion', 'nanotech', 'transportation', 'security'}


In [31]:
comp_df=pd.DataFrame(companies)

set(comp_df.category_code)

tech_companies=list(coll.find({"$and" : [{"$or":[{"category_code": 'web'},{'category_code':'analytics'},
                                                 {"category_code": 'software'},{"category_code": 'hardware'}]}, 
                                         {"offices.city": "New York"}]}))
len(tech_companies)

301

In [27]:
def getLocation(comp):
    longitude = comp['offices'][0]['longitude']
    latitude = comp['offices'][0]['latitude']
    loc = {
        'type':'Point',
        'coordinates':[(longitude), (latitude)]
    }
    return loc

In [28]:
for e in tech_companies:
    value={"$set" : {"loc": getLocation(e)}}
    coll.update_one(e,value)

In [39]:
#para crear nuevas collections
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["tech_companies"]
#para importar el contenido de un csv a mongo
#mongoimport --db companies --collection tech_companies --type csv --file Companies.csv --headerline

In [46]:
company_df=pd.DataFrame(tech_companies)

company_df.to_csv("Companies.csv")

In [32]:
coord=[]

for i in range(len(tech_companies)):
    coord.append(tech_companies[i]['loc']['coordinates'])

final_coord=[]

for e in coord:
    final_coord.append(e[::-1])

final_coord

[[47.603122, -122.333253],
 [40.757725, -73.986011],
 [40.72604, -73.995722],
 [37.393679, -122.079017],
 [40.736416, -73.993302],
 [40.7311317, -73.9919311],
 [None, None],
 [40.752957, -73.98546],
 [40.71014, -74.010808],
 [40.753385, -73.98962],
 [40.731118, -73.991933],
 [40.731118, -73.991933],
 [40.746953, -73.993553],
 [37.09024, -95.712891],
 [40.757929, -73.985506],
 [40.707921, -74.004792],
 [37.386665, -122.084171],
 [40.74406, -74.004592],
 [40.753202, -73.986771],
 [40.723703, -73.99644],
 [None, None],
 [None, None],
 [40.745216, -73.982807],
 [42.349164, -71.049346],
 [40.743808, -73.983626],
 [40.743808, -73.983626],
 [40.7550005, -73.9717364],
 [37.09024, -95.712891],
 [38.984172, -77.095965],
 [None, None],
 [None, None],
 [None, None],
 [37.09024, -95.712891],
 [40.740231, -73.985433],
 [40.737136, -73.987973],
 [40.757929, -73.985506],
 [40.762673, -73.969296],
 [40.754698, -73.992152],
 [37.402802, -122.117418],
 [37.09024, -95.712891],
 [40.797364, -73.97785],
 [4

In [33]:
comp_df=pd.DataFrame(final_coord)

comp_df

,0,1
0,47.603122,-122.333253
1,40.757725,-73.986011
2,40.726040,-73.995722
3,37.393679,-122.079017
4,40.736416,-73.993302
...,...,...
296,40.751804,-73.972365
297,NaN,NaN
298,40.726160,-74.006023
299,40.726160,-74.006023


In [34]:
comp_df=comp_df.dropna()

comp_df

,0,1
0,47.603122,-122.333253
1,40.757725,-73.986011
2,40.726040,-73.995722
3,37.393679,-122.079017
4,40.736416,-73.993302
...,...,...
294,40.702976,-74.014352
295,51.452512,-2.590974
296,40.751804,-73.972365
298,40.726160,-74.006023


In [35]:
lat=list(comp_df[0])

lng=list(comp_df[1])

coordinates_comp=list(zip(lat,lng))

coordinates_comp

[(47.603122, -122.333253),
 (40.757725, -73.986011),
 (40.72604, -73.995722),
 (37.393679, -122.079017),
 (40.736416, -73.993302),
 (40.7311317, -73.9919311),
 (40.752957, -73.98546),
 (40.71014, -74.010808),
 (40.753385, -73.98962),
 (40.731118, -73.991933),
 (40.731118, -73.991933),
 (40.746953, -73.993553),
 (37.09024, -95.712891),
 (40.757929, -73.985506),
 (40.707921, -74.004792),
 (37.386665, -122.084171),
 (40.74406, -74.004592),
 (40.753202, -73.986771),
 (40.723703, -73.99644),
 (40.745216, -73.982807),
 (42.349164, -71.049346),
 (40.743808, -73.983626),
 (40.743808, -73.983626),
 (40.7550005, -73.9717364),
 (37.09024, -95.712891),
 (38.984172, -77.095965),
 (37.09024, -95.712891),
 (40.740231, -73.985433),
 (40.737136, -73.987973),
 (40.757929, -73.985506),
 (40.762673, -73.969296),
 (40.754698, -73.992152),
 (37.402802, -122.117418),
 (37.09024, -95.712891),
 (40.797364, -73.97785),
 (40.75362, -73.986557),
 (40.707253, -74.007413),
 (40.715145, -74.005897),
 (40.75033, -73.

In [36]:
map_city = folium.Map(location=(40.7142700, -74.0059700), zoom_start=12)
map_city

In [37]:
for e in coordinates_comp:
    folium.Marker(e, radius=2,
                    icon=folium.Icon(prefix='fa',icon='cloud',color='red'), 
                   ).add_to(map_city)

In [38]:
map_city